In [1]:
from SimilarityClustering import SimilarityClustering
import articles_data

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import cnouns as cn
from pymongo import MongoClient
import datetime

%matplotlib inline

In [2]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('bclusters')
articles_collection = db.get_collection('barticles')

In [4]:
clusters_collection.delete_many({})
articles_collection.delete_many({})

In [4]:
datastore_dir = "../datastore/"
catelist_path = datastore_dir + "category2.p"

w2v_src_dir = datastore_dir + "w2v_src4"
w2v_path = datastore_dir + "sejongcorpus_w2v4_2.p"

nnp_dict_path = datastore_dir + "nnps2.p"
corpus_path = datastore_dir + "corpus2.p"

target_time = datetime.datetime(2016, 11, 15)
# target_time = datetime.datetime.now()
prefix = int("%.2d%.2d"%(target_time.month, target_time.day))
prefix_str = "%d_00" % prefix
prefix_str

'1115_00'

In [5]:
nnp_dict_df = pd.read_pickle(nnp_dict_path)
nnp_dict_df = nnp_dict_df[nnp_dict_df>10]
nnp_dict = nnp_dict_df.index.tolist()

custom_dict = [u'새누리', u'새누리당', u'더민주', u'더민주당', u'최순실', u'박대통령', u'국회의장', u'야권의요구', u'정기국회', u'참여정부']
dicts = set(nnp_dict + custom_dict)

def tokenizer(inp_str):
#     return cn.pos_tags(inp_str)
   return cn.custom_pos_tags(inp_str, dicts)

In [6]:
len(nnp_dict)

4656

# Doc2Vec

In [7]:
size = articles_data.makeDataset(crawled_collection, w2v_src_dir, corpus_path, tokenizer=tokenizer)
print size

Number of articles - 152728
Number of corpus - 1273272
Number of words - 1426000
Number of batchs - 286
Batch#0 - tokenizer took 33.838441 sec
Batch#1 - tokenizer took 2.497328 sec
Batch#2 - tokenizer took 2.663000 sec
Batch#3 - tokenizer took 2.139003 sec
Batch#4 - tokenizer took 2.217016 sec
Batch#5 - tokenizer took 2.371563 sec
Batch#6 - tokenizer took 2.374850 sec
Batch#7 - tokenizer took 3.257765 sec
Batch#8 - tokenizer took 2.955734 sec
Batch#9 - tokenizer took 2.542470 sec
Batch#10 - tokenizer took 2.188086 sec
Batch#11 - tokenizer took 2.948571 sec
Batch#12 - tokenizer took 2.573278 sec
Batch#13 - tokenizer took 3.022088 sec
Batch#14 - tokenizer took 3.126356 sec
Batch#15 - tokenizer took 2.610606 sec
Batch#16 - tokenizer took 2.654840 sec
Batch#17 - tokenizer took 2.354962 sec
Batch#18 - tokenizer took 4.096016 sec
Batch#19 - tokenizer took 2.844310 sec
Batch#20 - tokenizer took 2.833062 sec
Batch#21 - tokenizer took 3.312590 sec
Batch#22 - tokenizer took 2.290550 sec
Batch#23

In [8]:
articles_data.trainWord2Vec(w2v_src_dir, w2v_path, size)

NameError: name 'asd' is not defined

# Model

In [ ]:
train_df = articles_data.find_recent_articles(crawled_collection, catelist_path, target_time)
sc = SimilarityClustering()
sc.train("cate", w2v_path, train_df, path="../datastore", prefix=prefix_str, tokenizer=tokenizer,
            threshold=0.8, 
            cnt_threshold=2, 
            repeat=1,
            model_name='dbow+dmm')

In [ ]:
sc = SimilarityClustering.load(only_d2v=False, path="../datastore", prefix=prefix_str, model_name='dbow+dmm')

In [ ]:
sc.select_model('dbow+dmm')
sc.cluster_train("cate", path="../datastore", prefix=prefix_str, repeat=1, threshold=0.7)

# Save

In [ ]:
sc.save(path="../datastore", prefix=prefix_str)

In [ ]:
calced_clusters = sc.save_to_db(prefix, clusters_collection, articles_collection, target_time, test=True)

In [ ]:
# calced_clusters

# Test

In [ ]:
sc.iner_score(threshold=0.7, cnt_threshold=10)

In [ ]:
sc.print_clusters(top=15, sortby='similarity')

In [ ]:
cdf = pd.DataFrame(calced_clusters, columns=['cluster', 'portion', 'deltaTime', 'cohesion', 'ntc'])
sort_cdf = cdf.sort_values('ntc', ascending=False)
sort_cdf

In [ ]:
for idx, row in sort_cdf.iterrows():
    for c in calced_clusters:
        if(c['cluster'] == row.cluster):
            print c['ntc'], c['leading']['title']